# NYC Government Health Insurance AI Assistant
A Retrieval-Augmented Generation (RAG) System for Policy-Aligned Decision Support
### Project Overview
This notebook presents the end-to-end development of a Retrieval-Augmented Generation (RAG)–based AI system designed to assist NYC government employees in navigating official health insurance policies. The system delivers accurate, policy-grounded responses to benefits-related queries by retrieving and synthesizing information exclusively from authoritative source documents.
The project emphasizes technical rigor, responsible AI design, and practical business applicability, consistent with BUSA course objectives.
### Problem Statement
Health insurance policy documents are lengthy, complex, and difficult for employees to interpret during time-sensitive decision points such as enrollment, plan changes, and life events. Generic conversational AI systems pose a risk of hallucination, misinterpretation, and non-compliance in this regulated domain.
This project addresses the problem by implementing a controlled RAG architecture that prioritizes traceability, accuracy, and user trust.
Objectives (Grading-Aligned)
Design a policy-grounded question-answering system using RAG
Implement intent-aware query handling aligned with user decision flows
Minimize hallucinations through document-restricted generation and fallback logic
Demonstrate ethical AI practices in a regulated business context
Evaluate system performance through representative user queries
### Target Users & Use Case
NYC government employees (non-medical staff)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import userdata
from openai import OpenAI

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/health_chatbot_dataset.csv")
df.head()

,intent,user_query,ideal_response,notes,document
0,eligibility_and_qualifications,Am I eligible for health insurance through my ...,Eligibility varies by employment status and co...,Ground answers in retrieved policy text; avoid...,SECTION_I_EMPLOYEE_HEALTH_BENEFITS
1,eligibility_and_qualifications,Do I qualify for coverage if I work part-time?,Eligibility varies by employment status and co...,Ground answers in retrieved policy text; avoid...,SECTION_I_EMPLOYEE_HEALTH_BENEFITS
2,eligibility_and_qualifications,Can I add my spouse or partner to my plan?,Eligibility varies by employment status and co...,Ground answers in retrieved policy text; avoid...,SECTION_I_EMPLOYEE_HEALTH_BENEFITS
3,eligibility_and_qualifications,Are children over 18 eligible?,Eligibility varies by employment status and co...,Ground answers in retrieved policy text; avoid...,SECTION_I_EMPLOYEE_HEALTH_BENEFITS
4,eligibility_and_qualifications,Can I enroll if I recently changed jobs?,Eligibility varies by employment status and co...,Ground answers in retrieved policy text; avoid...,SECTION_I_EMPLOYEE_HEALTH_BENEFITS


## chunking strategy

In [ ]:
!pip -q install pymupdf

import fitz  # PyMuPDF
import re
import hashlib
from typing import List, Dict, Tuple
from collections import defaultdict

# ---------------------------
# 0) Scope mapping (DOCUMENT-LEVEL ROUTING)
# ---------------------------
SCOPE_MAP = {
    "SECTION_I_EMPLOYEE_HEALTH_BENEFITS": "employee",
    "SECTION_II_RETIREE_HEALTH_BENEFITS": "retiree",
    "SECTION_III_COBRA_Continuity_and_Qualifying_Events": "cobra",
    "SECTION_IV_SPECIAL_ENROLLMENT_PROVISION": "special_enrollment",
    "SECTION_V_Transgender_Care_and_Medical_Benefit_Standards": "medical_standards",
    "SECTION_VI_Surprise_Billing_and_Additional_Provisions": "surprise_billing",
    "SECTION_VII_Health_Plans_Overview_and_SBC_Details&Glossary": "plans_sbc_glossary",
    "SECTION_VIII_PICA_Prescription_and_senior_benefits": "pharmacy_pica",
    "SECTION_IX_EAP&blood_program": "eap_wellness",
}

def infer_scope_from_source(source_name: str) -> str:
    for k, v in SCOPE_MAP.items():
        if k in source_name:
            return v
    return "general"

# ---------------------------
# 1) Patterns
# ---------------------------
TABLE_START_PATTERNS = [
    r"^HEALTH PLANS\s*&\s*PICA PROGRAM FOR EMPLOYEES",
    r"^HEALTH PLAN\s+PLAN TYPE\s+PHONE NUMBER",
    r"^The following health plans are offered",
    r"^The following plans are approved Medicare HMOs",
]

NOISE_PATTERNS = [
    r"^page\s+\d+\s+of\s+\d+$",
    r"^\d+\s*$",
    r"^table of contents$",
    r"^contents$",
    r"^nycers\b",
]

# Topic hints (lightweight, deterministic)
TOPIC_PATTERNS = [
    ("eligibility", r"\beligib(?:le|ility)\b|\bwho is eligible\b|\bqualif(?:y|ication)\b"),
    ("enrollment", r"\benroll(?:ment|)\b|\bhow to enroll\b|\bsign up\b"),
    ("dependents", r"\bdependent(?:s|)\b|\bspouse\b|\bchild(?:ren)?\b|\bfamily\b"),
    ("claims_billing", r"\bclaim(?:s)?\b|\beob\b|\bbilling\b|\binvoice\b|\bpayment\b"),
    ("prior_auth", r"\bprior authorization\b|\bpre[- ]?approval\b"),
    ("cobra", r"\bcobra\b|\bcontinuation\b"),
    ("special_enrollment", r"\bspecial enrollment\b|\bqle\b|\bqualifying life event\b"),
    ("pharmacy", r"\bpharmacy\b|\bprescription\b|\bdrug\b|\bformulary\b"),
    ("surprise_billing", r"\bsurprise billing\b|\bbalance bill(?:ing)?\b|\bout[- ]of[- ]network\b"),
    ("eap_wellness", r"\beap\b|\bwellness\b|\bassistance program\b"),
    ("glossary", r"\bglossary\b|\bdefinitions\b"),
]

def is_noise_line(ln: str) -> bool:
    low = ln.strip().lower()
    return any(re.match(p, low) for p in NOISE_PATTERNS)

def is_heading(ln: str) -> bool:
    """
    Improved heading heuristic:
    - Supports ALL-CAPS headings
    - Supports numbered headings like 'I. ELIGIBILITY' or '1. ELIGIBILITY'
    - Avoids sentences
    """
    s = ln.strip()
    if len(s) < 4 or len(s) > 110:
        return False
    if s.endswith("."):
        return False

    # Reject if it looks like a sentence
    if re.search(r"\b(the|and|or|but|because|which|that)\b", s.lower()):
        # still allow if it is clearly all-caps
        pass

    # Numbered headings (common in handbooks)
    if re.match(r"^(\d+|[IVX]+)\.?\s+[A-Z]", s):
        return True

    letters = [c for c in s if c.isalpha()]
    if len(letters) < 4:
        return False

    upper_ratio = sum(c.isupper() for c in letters) / len(letters)
    # All-caps style
    if upper_ratio >= 0.85:
        # Avoid comma-heavy lines / table rows
        if s.count(",") >= 3:
            return False
        return True

    return False

def normalize_space(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def make_key(source: str, kind: str, title: str, occ: int, start_page=None, end_page=None) -> str:
    safe_title = normalize_space(title)[:60]
    page_part = f"p{start_page}-p{end_page}::" if start_page is not None else ""
    return f"{source}::{kind}::{page_part}{safe_title}::{occ:03d}"

def fingerprint(txt: str) -> str:
    norm = normalize_space(txt.lower())
    return hashlib.md5(norm.encode("utf-8")).hexdigest()

def dedupe_chunks(chunks: List[Dict]) -> List[Dict]:
    seen = set()
    out = []
    for c in chunks:
        fp = fingerprint(c["text"])
        if fp in seen:
            continue
        seen.add(fp)
        out.append(c)
    return out

def infer_topic(text: str) -> str:
    t = text.lower()
    for topic, pat in TOPIC_PATTERNS:
        if re.search(pat, t, flags=re.IGNORECASE):
            return topic
    return "general"

def infer_chunk_type(kind: str, title: str, text: str) -> str:
    """
    Upgrade chunk_type deterministically when the content is an eligibility rule block.
    """
    if kind == "plan_catalog":
        return "plan_catalog"
    low = (title + "\n" + text).lower()
    if re.search(r"\beligib(?:le|ility)\b|\bwho is eligible\b|\bqualif(?:y|ication)\b", low):
        return "eligibility_rule"
    return "section"

# ---------------------------
# 2) Extraction (PAGE-AWARE)
# ---------------------------
def extract_pdf_lines_with_pages(pdf_path: str) -> List[Tuple[int, str]]:
    doc = fitz.open(pdf_path)
    out: List[Tuple[int, str]] = []
    for i, page in enumerate(doc):
        page_no = i + 1  # 1-indexed for citations
        text = page.get_text("text")
        for ln in text.splitlines():
            ln = ln.strip()
            if not ln:
                continue
            if is_noise_line(ln):
                continue
            out.append((page_no, ln))
    return out

# ---------------------------
# 3) Chunking (STRUCTURE + METADATA)
# ---------------------------
def chunk_pdf(pdf_path: str, source_name: str = None) -> List[Dict]:
    source_name = source_name or pdf_path.split("/")[-1].replace(".pdf", "")
    scope = infer_scope_from_source(source_name)
    lines = extract_pdf_lines_with_pages(pdf_path)

    chunks: List[Dict] = []
    buf: List[Tuple[int, str]] = []
    current_title = "INTRO"
    current_start_page = None

    in_plan_table = False
    table_buf: List[Tuple[int, str]] = []

    title_counter = defaultdict(int)

    def flush(title: str, content_lines: List[Tuple[int, str]], kind: str = "section"):
        if not content_lines:
            return
        # build text + page range
        pages = [p for p, _ in content_lines]
        start_p, end_p = min(pages), max(pages)
        text = "\n".join(ln for _, ln in content_lines).strip()
        if not text:
            return

        topic = infer_topic(text)
        chunk_type = infer_chunk_type(kind, title, text)

        title_counter[(kind, title, start_p, end_p)] += 1
        occ = title_counter[(kind, title, start_p, end_p)]

        chunks.append({
            "key": make_key(source_name, chunk_type, title, occ, start_page=start_p, end_page=end_p),
            "text": text,
            "source": source_name,
            "section": title,
            "scope": scope,
            "topic": topic,
            "chunk_type": chunk_type,
            "page_start": start_p,
            "page_end": end_p,
        })

    for page_no, ln in lines:
        # --- Plan table start detection
        if any(re.match(pat, ln, re.IGNORECASE) for pat in TABLE_START_PATTERNS):
            flush(current_title, buf, kind="section")
            buf = []
            in_plan_table = True
            table_buf = [(page_no, ln)]
            current_title = "PLAN_CATALOG"
            current_start_page = page_no
            continue

        # --- If inside plan table, absorb until a strong heading indicates table end
        if in_plan_table:
            if is_heading(ln) and len(table_buf) > 6 and not re.match(r"^Health Plan\b", ln, re.IGNORECASE):
                flush("PLAN_CATALOG", table_buf, kind="plan_catalog")
                table_buf = []
                in_plan_table = False
                # fall through to process ln below
            else:
                table_buf.append((page_no, ln))
                continue

        # --- Heading-based chunking
        if is_heading(ln):
            if normalize_space(ln) == normalize_space(current_title):
                continue
            flush(current_title, buf, kind="section")
            current_title = ln
            buf = [(page_no, ln)]
            current_start_page = page_no
        else:
            buf.append((page_no, ln))

    # Final flushes
    if in_plan_table and table_buf:
        flush("PLAN_CATALOG", table_buf, kind="plan_catalog")
    flush(current_title, buf, kind="section")

    return chunks

# ---------------------------
# 4) Batch run (your 9 PDFs)
# ---------------------------
pdf_paths = [
    "/content/drive/MyDrive/Rag_documents /SECTION_ III_COBRA_Continuity_and_Qualifying_Events.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_II_RETIREE_HEALTH_BENEFITS.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_IV_SPECIAL_ENROLLMENT_PROVISION.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_IX_EAP&blood_program.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_I_EMPLOYEE_HEALTH_BENEFITS.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_V_Transgender_Care_and_Medical_Benefit_Standards.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_VI_Surprise_Billing_and_Additional_Provisions.pdf.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_VII_Health_Plans_Overview_and_SBC_Details&Glossary.pdf",
    "/content/drive/MyDrive/Rag_documents /SECTION_VIII_PICA_Prescription_and_senior_benefits.pdf"
]

all_chunks: List[Dict] = []
failed = []

for path in pdf_paths:
    try:
        chunks = chunk_pdf(path)
        all_chunks.extend(chunks)
        elig = sum(1 for c in chunks if c["chunk_type"] == "eligibility_rule")
        print(f"✓ {path.split('/')[-1]} -> {len(chunks)} chunks | eligibility_rule={elig}")
    except Exception as e:
        failed.append((path, str(e)))
        print(f"✗ Failed: {path.split('/')[-1]} -> {e}")

before = len(all_chunks)
all_chunks = dedupe_chunks(all_chunks)
after = len(all_chunks)

print(f"\n✓ Created {before} chunks (raw) from {len(pdf_paths)} documents")
print(f"✓ After dedupe: {after} chunks")
print("Sample keys:", [c["key"] for c in all_chunks[:5]])

if failed:
    print("\nFailures:")
    for p, err in failed:
        print("-", p, "->", err)

✓ SECTION_ III_COBRA_Continuity_and_Qualifying_Events.pdf -> 21 chunks | eligibility_rule=11
✓ SECTION_II_RETIREE_HEALTH_BENEFITS.pdf -> 37 chunks | eligibility_rule=18
✓ SECTION_IV_SPECIAL_ENROLLMENT_PROVISION.pdf -> 16 chunks | eligibility_rule=6
✓ SECTION_IX_EAP&blood_program.pdf -> 7 chunks | eligibility_rule=4
✓ SECTION_I_EMPLOYEE_HEALTH_BENEFITS.pdf -> 53 chunks | eligibility_rule=24
✓ SECTION_V_Transgender_Care_and_Medical_Benefit_Standards.pdf -> 20 chunks | eligibility_rule=3
✓ SECTION_VI_Surprise_Billing_and_Additional_Provisions.pdf.pdf -> 11 chunks | eligibility_rule=0
✓ SECTION_VII_Health_Plans_Overview_and_SBC_Details&Glossary.pdf -> 67 chunks | eligibility_rule=4
✓ SECTION_VIII_PICA_Prescription_and_senior_benefits.pdf -> 56 chunks | eligibility_rule=13

✓ Created 288 chunks (raw) from 9 documents
✓ After dedupe: 279 chunks
Sample keys: ['SECTION_ III_COBRA_Continuity_and_Qualifying_Events::section::p1-p1::INTRO::001', 'SECTION_ III_COBRA_Continuity_and_Qualifying_Events


# What this code does (short):
# - Loads each PDF using PyMuPDF and extracts text line-by-line.
# - Filters common PDF noise (headers/footers/page numbers/TOC) to reduce junk chunks.
# - Chunks content by detecting strong ALL-CAPS section headings (semantic chunking).
# - Detects “plan catalog” table regions (where present) and stores them as dedicated
#   plan_catalog chunks to support “List all plans” style questions.
# - Generates unique, stable chunk keys with occurrence counters and removes exact
#   duplicate chunks via content fingerprinting before embedding.

# What’s needed to move from class-grade to production-grade:
# - Stronger PDF structure parsing:
#   * Use table extraction (e.g., Camelot/Tabula) or layout-aware parsing to preserve
#     true table rows/columns; avoid relying only on text lines.
# - Metadata hardening:
#   * Store page ranges (start_page/end_page), document version/date, and section hierarchy
#     (doc -> section -> subsection) for traceability and debugging.
# - Heading normalization + hierarchy:
#   * Normalize near-duplicate headings (e.g., “MEDICARE & …” vs “MEDICARE …”) and track
#     heading levels to avoid fragmented or redundant sections.
# - Quality gates:
#   * Add automated checks (min/max chunk length, overlap rules, %duplicate threshold,
#     empty/boilerplate detection) and fail fast if chunk quality is poor.
# - Retrieval optimization:
#   * Use hybrid retrieval (BM25 + embeddings), reranking, and intent-aware retrieval
#     rules (e.g., always prioritize plan_catalog for plan-options intent).
# - Governance and safety:
#   * Enforce source-grounded responses, citation mapping (chunk_id -> text span), and
#     safe fallback when the retrieved context does not support a definitive answer.
# - Monitoring and iteration:
#   * Log queries, retrieved chunk_ids, similarity scores, and user feedback to drive
#     continuous improvements in chunking and retrieval performance.


In [ ]:
import re
from typing import List, Dict, Tuple

PHONE_RE = re.compile(
    r'(\(?\d{3}\)?[\s\-\.]?\d{3}[\s\-\.]?\d{4})'  # 212-555-1212, (212) 555-1212, etc.
)

# Common plan-type tokens seen in benefit docs
PLAN_TYPE_TOKENS = [
    "EPO", "PPO", "POS", "HMO", "HDHP", "CDHP", "HSA", "HRA"
]

def _normalize_lines(text: str) -> List[str]:
    lines = []
    for ln in text.splitlines():
        ln = re.sub(r'\s+', ' ', ln.strip())
        if ln:
            lines.append(ln)
    return lines

def _looks_like_table_header(ln: str) -> bool:
    header_hits = [
        "health plan",
        "plan type",
        "phone",
        "phone number",
        "medicare",
        "approved medicare"
    ]
    low = ln.lower()
    return any(h in low for h in header_hits)

def _strip_trailing_nonname(s: str) -> str:
    # Remove trailing plan-type tokens, phone fragments, and obvious table noise
    s = re.sub(PHONE_RE, "", s).strip()
    # remove trailing tokens like "EPO", "PPO"
    for tok in PLAN_TYPE_TOKENS:
        s = re.sub(rf"\b{tok}\b$", "", s).strip()
    # cleanup leftover separators
    s = s.strip(" -–—|•\t")
    return s.strip()

def extract_plan_names_from_catalog(plan_catalog_text: str) -> List[str]:
    """
    Extract plan names from a plan-catalog chunk that was captured as one chunk.
    Heuristic: a table "row" ends when a phone number appears. Everything before the phone
    is treated as the row content, and the plan name is inferred from the leading segment.
    """
    lines = _normalize_lines(plan_catalog_text)

    plans = []
    seen = set()

    row_buf = ""

    for ln in lines:
        # skip obvious headers
        if _looks_like_table_header(ln):
            continue

        # Accumulate into a "row buffer" because PDF tables wrap lines
        if row_buf:
            row_buf = f"{row_buf} {ln}"
        else:
            row_buf = ln

        # If we have a phone, we assume the row is complete
        if PHONE_RE.search(row_buf):
            # Example row_buf patterns:
            # "Aetna EPO EPO 1-800-xxx-xxxx"
            # "HIP Prime POS POS (212) xxx-xxxx"
            # "NYC Medicare Advantage Plan HMO 1-xxx-xxx-xxxx"
            row_wo_phone = PHONE_RE.sub("", row_buf).strip()

            # Try to split on plan type tokens if present
            name_candidate = row_wo_phone
            for tok in PLAN_TYPE_TOKENS:
                # Split at the first occurrence of a plan type token
                m = re.search(rf"\b{tok}\b", row_wo_phone)
                if m:
                    name_candidate = row_wo_phone[:m.start()].strip()
                    break

            name_candidate = _strip_trailing_nonname(name_candidate)

            # Guardrails: ignore tiny fragments
            if len(name_candidate) >= 4 and not _looks_like_table_header(name_candidate):
                if name_candidate not in seen:
                    plans.append(name_candidate)
                    seen.add(name_candidate)

            # reset for next row
            row_buf = ""

    # If anything left in buffer without phone, we discard (not a full row)
    return plans


In [ ]:
def categorize_plan_list(catalog_text: str) -> Tuple[str, List[str]]:
    """
    Returns (category, plans)
    """
    low = catalog_text.lower()
    if "approved medicare" in low or "medicare hmo" in low or "medicare advantage" in low:
        category = "medicare"
    else:
        category = "employee_non_medicare"
    return category, extract_plan_names_from_catalog(catalog_text)


In [ ]:
def answer_plan_options(top_chunks: List[dict]) -> str:
    """
    top_chunks: list of retrieved chunks (dicts) including at least one plan_catalog chunk.
    Expected keys: text, chunk_type (ideally), source
    """
    catalogs = [c for c in top_chunks if c.get("chunk_type") == "plan_catalog"]
    if not catalogs:
        # fallback: try all chunks (last resort)
        catalogs = top_chunks[:3]

    bucket = {"employee_non_medicare": [], "medicare": []}

    for c in catalogs:
        category, plans = categorize_plan_list(c["text"])
        bucket[category].extend(plans)

    # de-dup while preserving order
    def uniq(seq):
        seen = set()
        out = []
        for x in seq:
            if x not in seen:
                out.append(x)
                seen.add(x)
        return out

    emp = uniq(bucket["employee_non_medicare"])
    med = uniq(bucket["medicare"])

    if not emp and not med:
        return (
            "I couldn’t reliably extract a complete plan list from the retrieved policy text. "
            "This usually means the plan table wasn’t retrieved as a single chunk. "
            "Re-run chunking with a dedicated PLAN_CATALOG chunk, then retry."
        )

    parts = []
    if emp:
        parts.append("**Employee / Non-Medicare plan options (from the plan catalog):**\n- " + "\n- ".join(emp))
    if med:
        parts.append("**Medicare plan options (from the plan catalog):**\n- " + "\n- ".join(med))

    return "\n\n".join(parts)


# NOTES:
 This step prepares the final RAG inputs by separating the chunk text (used for embedding)
 from the chunk identifiers and metadata (used for retrieval traceability).
 Each embedded vector is mapped back to a unique chunk key, allowing retrieved responses
 to be traced to the originating PDF and section.
#
 For production systems, chunk metadata (source document, section, chunk type, page range)should be stored alongside each vector to support citation, debugging, and intent-awareretrieval. For this academicimplementation, keys provide sufficient linkage between embeddings and source content.


In [ ]:
rag_texts = [chunk["text"] for chunk in all_chunks]

rag_metadata = [
    {
        "key": chunk["key"],
        "source": chunk["source"],
        "section": chunk["section"],
        "chunk_type": chunk["chunk_type"]
    }
    for chunk in all_chunks
]

print(f"✓ Created rag_texts with {len(rag_texts)} chunks")
print(f"✓ Created rag_metadata with {len(rag_metadata)} entries")


✓ Created rag_texts with 279 chunks
✓ Created rag_metadata with 279 entries


# RAG_EMBEDDING

 since we have #238 chunks from semantic chunking we will now embed this
 ### This step converts each section-level chunk into a dense vector embedding using OpenAI’s text-embedding-3-large model. These embeddings capture semantic meaning and enable similarity-based retrieval during inference.

 ### Each embedding corresponds one-to-one with a chunk produced by the section-based chunking strategy, allowing the system to retrieve entire policy sections rather than fragmented text. This improves answer grounding and interpretability.


In [ ]:
def embed(texts):
    r = client.embeddings.create(
        model="text-embedding-3-large",
        input=texts
    )
    return [np.array(x.embedding, dtype="float32") for x in r.data]

embeddings = embed(rag_texts)


# Retrival startegy

In [ ]:
import numpy as np

def l2_normalize(v: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(v)
    return v / (n + eps)

# Pre-normalize once (important for speed and consistency)
emb_matrix = np.vstack([e for e in embeddings]).astype("float32")
emb_matrix = emb_matrix / (np.linalg.norm(emb_matrix, axis=1, keepdims=True) + 1e-12)

def retrieve_with_scores(query: str, k: int = 5, min_score: float = 0.25):
    q = embed_batched([query], batch_size=1)[0].astype("float32")
    q = l2_normalize(q).astype("float32")

    # Cosine similarity becomes dot product after normalization
    sims = emb_matrix @ q  # shape: (num_chunks,)
    top_idx = np.argsort(-sims)[:k]

    results = []
    for i in top_idx:
        score = float(sims[i])
        if score < min_score:
            continue
        results.append((score, rag_keys[i], rag_texts[i]))
    return results


In [ ]:
rag_meta = [
    {"key": c["key"], "source": c["source"], "section": c["section"], "chunk_type": c["chunk_type"]}
    for c in all_chunks
]


In [ ]:
def embed_batched(texts, batch_size=96):
    vectors = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        r = client.embeddings.create(
            model="text-embedding-3-large",
            input=batch
        )
        vectors.extend([np.array(x.embedding, dtype="float32") for x in r.data])
    return vectors


In [ ]:
def retrieve_insurance(query: str, k: int = 5, min_score: float = 0.25, boost_plan_catalog: float = 0.08):
    q = embed_batched([query], batch_size=1)[0].astype("float32")
    q = l2_normalize(q).astype("float32")
    sims = emb_matrix @ q

    # Lightweight query intent heuristic (class-safe, no extra model needed)
    qlow = query.lower()
    is_plan_options = any(p in qlow for p in ["plan options", "plans available", "which plans", "list all plans", "plan choices"])

    if is_plan_options:
        # Add a small score boost to plan_catalog chunks so they rank above plan detail chunks
        for i, m in enumerate(rag_meta):
            if m.get("chunk_type") == "plan_catalog":
                sims[i] += boost_plan_catalog

    top_idx = np.argsort(-sims)[:k]

    results = []
    for i in top_idx:
        score = float(sims[i])
        if score < min_score:
            continue
        results.append({
            "score": score,
            "key": rag_keys[i],
            "text": rag_texts[i],
            "meta": rag_meta[i]
        })
    return results


In [ ]:
def rerank_diverse(results, k=5):
    picked = []
    seen_sections = set()
    for r in results:
        sec = r["meta"].get("section", "")
        if sec in seen_sections:
            continue
        picked.append(r)
        seen_sections.add(sec)
        if len(picked) == k:
            break
    return picked


In [ ]:
def retrieval_confidence(results, strong=0.35, weak=0.25):
    if not results:
        return "none"
    top = results[0]["score"]
    if top >= strong:
        return "strong"
    if top >= weak:
        return "weak"
    return "none"


In [ ]:
def retrieve_pipeline(query: str, k: int = 5):
    results = retrieve_insurance(query, k=12, min_score=0.22)
    results = rerank_diverse(results, k=k)
    mode = retrieval_confidence(results)
    return results, mode


In [ ]:
# CREATE rag_texts and rag_keys
rag_texts = [chunk['text'] for chunk in all_chunks]
rag_keys = [chunk['key'] for chunk in all_chunks]

print(f"\n✓ Created rag_texts with {len(rag_texts)} chunks")
print(f"✓ Created rag_keys with {len(rag_keys)} keys")


✓ Created rag_texts with 279 chunks
✓ Created rag_keys with 279 keys



Retrieval Method Summary
* Document retrieval is performed using cosine similarity between the user query embedding and section-level document embeddings.
* All embeddings are L2-normalized to ensure stable and comparable similarity scores.
* A minimum similarity threshold is applied to prevent retrieving irrelevant policy sections and to support safe fallback behavior.
Retrieval is intent-aware:
* For plan-selection queries, chunks labeled as plan_catalog are prioritized to ensure complete plan listings are retrieved.
* Chunk metadata (source document, section title, and chunk type) is used to improve retrieval precision and traceability.
* A diversity constraint is applied so that multiple chunks from the same section do not dominate the top-k results.
Retrieval confidence is classified as strong, weak, or none, enabling controlled response generation and reduced hallucination risk.

# checking the rag coverage for the intent map

In [ ]:

# Making a dataframe of the intent dataset
intent_df = pd.read_csv("/content/drive/MyDrive/health_chatbot_dataset.csv")

In [ ]:
intent_df = pd.read_csv("/content/drive/MyDrive/health_chatbot_dataset.csv")

def test_intent_coverage(intent_df, threshold=0.40, k=5): # Added 'k' parameter with a default value
    rows = []

    for _, row in intent_df.iterrows():
        intent = row["intent"]
        query  = row["user_query"]

        results, mode = retrieve_pipeline(query, k=k) # Pass k to retrieve_pipeline

        if results:
            top_score = results[0]["score"]
            top_key   = results[0]["key"]
        else:
            top_score = 0.0
            top_key   = "NO_RETRIEVAL"

        fallback_needed = (mode == "none") or (top_score < threshold)

        rows.append([
            intent,
            query,
            round(top_score, 3),
            top_key,
            mode,
            fallback_needed
        ])

    return pd.DataFrame(
        rows,
        columns=["Intent", "Query", "Top Score", "Top Chunk", "Retrieval Mode", "Fallback Needed"]
    )

df = test_intent_coverage(intent_df, threshold=0.40)
df

,Intent,Query,Top Score,Top Chunk,Retrieval Mode,Fallback Needed
0,eligibility_and_qualifications,Am I eligible for health insurance through my ...,0.508,SECTION_I_EMPLOYEE_HEALTH_BENEFITS::eligibilit...,strong,False
1,eligibility_and_qualifications,Do I qualify for coverage if I work part-time?,0.453,SECTION_II_RETIREE_HEALTH_BENEFITS::eligibilit...,strong,False
2,eligibility_and_qualifications,Can I add my spouse or partner to my plan?,0.480,SECTION_II_RETIREE_HEALTH_BENEFITS::eligibilit...,strong,False
3,eligibility_and_qualifications,Are children over 18 eligible?,0.529,SECTION_II_RETIREE_HEALTH_BENEFITS::eligibilit...,strong,False
4,eligibility_and_qualifications,Can I enroll if I recently changed jobs?,0.500,SECTION_II_RETIREE_HEALTH_BENEFITS::section::p...,strong,False
...,...,...,...,...,...,...
75,pica_program_and_senior_prescription_benefits,Do I need a separate prescription drug plan if...,0.592,SECTION_VIII_PICA_Prescription_and_senior_bene...,strong,False
76,employee_assistance_and_wellness_programs,Does my employer offer an Employee Assistance ...,0.578,SECTION_IX_EAP&blood_program::eligibility_rule...,strong,False
77,employee_assistance_and_wellness_programs,What types of counseling or support services a...,0.561,SECTION_IX_EAP&blood_program::eligibility_rule...,strong,False
78,employee_assistance_and_wellness_programs,Is participation in wellness or blood donation...,0.571,SECTION_IX_EAP&blood_program::eligibility_rule...,strong,False


In [ ]:
#for t in [0.35, 0.45, 0.55, 0.65]:
  #  df_t = test_intent_coverage(intent_df, threshold=t, k=3)
   # pass_rate = (df_t["Fallback Needed"] == False).mean()
   # print(f"threshold={t:.2f} -> pass_rate={pass_rate:.1%}")
#uncomment to see how the threshhold affects the pass rate


Threshold sensitivity analysis was conducted to validate retrieval confidence gating.
At thresholds 0.30 and 0.35, the retriever produced adequate document support for 100% of test queries.
At 0.40, coverage remained high (95%), while still filtering out weaker matches.
At 0.45, coverage dropped to 85%, indicating an overly strict gate for interactive use.

Based on this trade-off, we select 0.40 as the evaluation threshold (rigor-focused),
and 0.35 for the live demo (user-experience focused) to minimize unnecessary fallbacks.


# Gradio

In [ ]:
import re

# =========================================================
# STRICT FALLBACK MESSAGE (define once, keep deterministic)
# =========================================================
STRICT_FALLBACK_MSG = (
    "I can’t recommend a specific plan for you. I can, however, compare plans using objective criteria "
    "(premium, deductible, copays, network, prescriptions) if you share the plan names you’re considering "
    "and whether this is open enrollment or a qualifying life event."
)

# =========================================================
# 0) NORMALIZATION + DETECTORS
# =========================================================
def normalize_query(q: str) -> str:
    q = q.lower()
    q = re.sub(r"[^a-z\s]", " ", q)
    q = re.sub(r"\s+", " ", q).strip()
    return q

def is_personal_recommendation(q: str) -> bool:
    qn = normalize_query(q)

    # Direct decision-phrases (high precision)
    decision_phrases = [
        "which plan should i", "what plan should i",
        "what is the best plan", "whats the best plan", "what's the best plan",
        "whats a good plan", "what's a good plan",
        "which plan is best", "which is better",
        "recommend a plan", "suggest a plan", "pick a plan", "choose a plan",
        "best insurance plan", "good insurance plan"
    ]

    # Broader signals (balanced recall)
    recommend_like = ["recommend", "reccomend", "reccommend", "suggest", "pick", "choose", "best", "better", "good"]
    plan_like = ["plan", "plans", "option", "options", "health plan", "insurance", "coverage"]
    personal_like = ["for me", " me ", " my ", " personally", " my needs", " for my "]

    if any(p in qn for p in decision_phrases):
        return True

    if any(r in qn for r in recommend_like) and any(pl in qn for pl in plan_like):
        return True

    if any(pl in qn for pl in plan_like) and any(pr in qn for pr in personal_like) and (
        "recommend" in qn or "suggest" in qn or "best" in qn or "good" in qn
    ):
        return True

    return False

def detect_affirmation(q: str) -> bool:
    qn = normalize_query(q)
    return qn in {"yes", "y", "yeah", "yep", "sure", "ok", "okay", "correct", "right"}


# =========================================================
# 0) STATE-AWARE ASK-BACK PLANNER (returns slot + question)
# =========================================================
def plan_missing_slot(query: str):
    q = query.lower()

    if "copay" in q and not any(x in q for x in ["aetna","anthem","hip","gk","nyc","ppo","epo","hmo"]):
        return ("awaiting_plan_name",
                "Which plan are you enrolled in (or comparing)? (e.g., Aetna EPO, Anthem PPO, HIP HMO)")

    if any(x in q for x in ["add", "enroll", "include"]) and any(x in q for x in ["child", "dependent", "son", "daughter"]) \
       and not any(x in q for x in ["birth", "adoption", "qualifying", "life event", "open enrollment"]):
        return ("awaiting_timing",
                "Is this during open enrollment, or a qualifying life event (birth/adoption)? What is the event date?")

    return (None, None)


# ----------------------------
# 1) EVIDENCE BUILDER (schema-safe)
# ----------------------------
def build_evidence(results, max_evidence=2, max_lines=2):
    ev = []
    for r in results[:max_evidence]:
        lines = [ln.strip() for ln in r.get("text", "").splitlines() if ln.strip()]
        snippet = " ".join(lines[:max_lines])

        meta = r.get("meta", {}) or {}
        key = r.get("key") or meta.get("key") or r.get("source") or meta.get("source") or "unknown_chunk"

        ev.append(f"- {key} (score={r.get('score', 0.0):.3f}): {snippet[:220]}")
    return "\n".join(ev)


# ----------------------------
# 2) PLAN OPTIONS DETECTION + EXTRACTION
# ----------------------------
def _chunk_type(r: dict) -> str:
    meta = r.get("meta", {}) or {}
    return r.get("chunk_type") or meta.get("chunk_type") or ""

def is_plan_options_query(q: str) -> bool:
    q = q.lower()
    phrases = [
        "plan options", "plans available", "list all plans", "which plans", "plan choices",
        "available plans", "what plans", "plan list", "options available"
    ]
    return any(p in q for p in phrases)

def answer_plan_options_deterministic(results):
    catalogs = [r for r in results if _chunk_type(r) == "plan_catalog"]
    if not catalogs:
        return None
    combined = "\n".join([c.get("text", "") for c in catalogs])
    plans = extract_plan_names_from_catalog(combined)  # <-- must exist in your notebook
    if not plans:
        return None
    return "**Available plans found in the documents:**\n- " + "\n- ".join(plans)


def answer_with_intelligent_fallback_insurance_v2_from_results(query, results, threshold=0.40):
    # STRICT FALLBACK FIRST (personal recommendation)
    if is_personal_recommendation(query):
        return STRICT_FALLBACK_MSG, "", 0.0, "Strict Fallback (Personal Recommendation)", None

    # STRICT FALLBACK (other out-of-scope)
    if is_out_of_scope(query):
        return STRICT_FALLBACK_MSG, "", 0.0, "Strict Fallback (Out of Scope)", None

    # STATE-AWARE ASK-BACK
    pending_slot, followup = plan_missing_slot(query)
    if followup:
        return followup, "", 0.0, "Ask-Back (Missing Details)", pending_slot

    # Score + evidence
    top_score = results[0]["score"] if results else 0.0
    evidence = build_evidence(results)

    # Deterministic plan extraction
    if is_plan_options_query(query):
        extracted = answer_plan_options_deterministic(results)
        if extracted:
            return extracted, evidence, top_score, "Plan Catalog Extraction", None

    # Confidence gate → intelligent fallback
    if top_score < threshold:
        return (
            "I can’t confirm that from the policy documents I have indexed. "
            "If you share your plan name (and timing: open enrollment vs qualifying event), I can narrow it down.",
            evidence,
            top_score,
            "Intelligent Fallback Used",
            None
        )

    # RAG answer (single LLM call, evidence-grounded)
    messages = [
        {
            "role": "system",
            "content": (
                "You are an NYC Employee Health Insurance Assistant.\n"
                "Use ONLY the provided evidence.\n"
                "Never invent policy details or numbers.\n"
                "If evidence is insufficient, ask exactly ONE targeted follow-up.\n"
                "Keep the answer concise."
            )
        },
        {
            "role": "user",
            "content": (
                f"EVIDENCE:\n{evidence}\n\n"
                f"TOP_SIMILARITY_SCORE: {top_score:.3f}\n\n"
                f"USER_QUESTION: {query}\n\n"
                "TASK:\n"
                "- Answer using evidence only.\n"
                "- If missing key details, ask exactly ONE targeted follow-up question."
            )
        }
    ]

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.2,
        max_tokens=220
    )
    ans = resp.choices[0].message.content
    return ans, evidence, top_score, "RAG Answer", None

# ----------------------------
# 4) CHAT FUNCTION (NOW STATEFUL; 5 outputs incl. state)
# ----------------------------
def chat_fn_intelligent(q, state):
    state = state or {"last_question": "", "pending_slot": None}

    # STRICT FALLBACK BEFORE ANYTHING
    if is_personal_recommendation(q):
        return STRICT_FALLBACK_MSG, "", f"{0.0:.3f}", "Strict Fallback (Personal Recommendation)", state

    # If user says "yes" while we’re waiting for a slot, ask for that slot explicitly
    if detect_affirmation(q) and state.get("pending_slot"):
        slot = state["pending_slot"]
        if slot == "awaiting_plan_name":
            return (
                "Please share the plan name (e.g., Aetna EPO, Anthem PPO, HIP HMO).",
                "",
                f"{0.0:.3f}",
                "Ask-Back (Plan Name Needed)",
                state
            )
        if slot == "awaiting_timing":
            return (
                "Please confirm: open enrollment or qualifying life event? If life event, what is the date?",
                "",
                f"{0.0:.3f}",
                "Ask-Back (Timing Needed)",
                state
            )

    # Retrieval
    results = retrieve_insurance(q, k=4, min_score=0.22)  # <-- must exist in your notebook

    # Main logic (returns pending_slot)
    ans, evidence, score, mode, pending_slot = answer_with_intelligent_fallback_insurance_v2_from_results(
        q, results, threshold=0.40
    )

    # Update state
    state["last_question"] = q
    state["pending_slot"] = pending_slot

    return ans, evidence, f"{score:.3f}", mode, state

In [ ]:
print(answer_with_intelligent_fallback_insurance_v2("Which plan is best for me?")[3])

RAG Answer


In [ ]:
summary = (df.assign(Passed=lambda x: ~x["Fallback Needed"])
             .groupby("Intent")
             .agg(total=("Query","count"),
                  pass_rate=("Passed","mean"),
                  avg_score=("Top Score","mean"))
             .sort_values(["pass_rate","avg_score"]))
summary


,total,pass_rate,avg_score
Intent,,,
prior_authorization_and_pre_approvals,4,0.75,0.497000
claims_eobs_and_billing_issues,5,0.80,0.416800
plan_options_and_comparison,5,0.80,0.528600
life_events_and_dependent_changes,4,1.00,0.478750
cost_sharing_copays_deductibles_oop_max,5,1.00,0.483400
eligibility_and_qualifications,5,1.00,0.494000
enrollment_and_required_documents,5,1.00,0.499000
provider_and_network_search,4,1.00,0.499000
coverage_and_benefits_overview,6,1.00,0.531333


Intent-level retrieval analysis shows strong document coverage across most policy areas.
The majority of intents achieved a 100% pass rate with average similarity scores above 0.50,
indicating reliable semantic alignment between user queries and policy sections.

A small number of intents—such as prior authorization, claims, and cost-sharing—show slightly
lower pass rates (75–80%), reflecting the higher variability and plan-specific nature of these topics.
These cases are appropriately routed to fallback behavior, demonstrating effective confidence gating.


In [ ]:
from google.colab import userdata
from openai import OpenAI

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input="This is a test of the text to speech system."
)

response.stream_to_file("test.mp3")


/tmp/ipython-input-4278194468.py:7: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("test.mp3")


In [ ]:
import gradio as gr
import re

# =========================================================
# 0) STRICT FALLBACK (Personal Recommendation) + Helpers
# =========================================================
STRICT_FALLBACK_MSG = (
    "I can’t recommend a plan or determine which option is best for you personally. "
    "Personalized recommendations are out of scope for this assistant. "
    "If you share the specific plans you’re comparing, I can provide an objective, policy-based comparison "
    "(copays, deductibles, networks) using only the official documents."
)

def normalize_query(q: str) -> str:
    q = q.lower()
    q = re.sub(r"[^a-z\s]", " ", q)
    q = re.sub(r"\s+", " ", q).strip()
    return q

def is_personal_recommendation(q: str) -> bool:
    qn = normalize_query(q)

    # More robust than your old one
    decision_phrases = [
        "which plan should i", "what plan should i",
        "what is the best plan", "whats the best plan", "what's the best plan",
        "whats a good plan", "what's a good plan",
        "which plan is best", "which is better",
        "recommend a plan", "suggest a plan", "pick a plan", "choose a plan",
        "best insurance plan", "good insurance plan"
    ]
    recommend_like = ["recommend", "reccomend", "reccommend", "suggest", "pick", "choose", "best", "better", "good"]
    plan_like = ["plan", "plans", "option", "options", "health plan", "insurance", "coverage"]
    personal_like = ["for me", " me ", " my ", " personally", " my needs", " for my "]

    if any(p in qn for p in decision_phrases):
        return True
    if any(r in qn for r in recommend_like) and any(pl in qn for pl in plan_like):
        return True
    if any(pl in qn for pl in plan_like) and any(pr in qn for pr in personal_like) and (
        "recommend" in qn or "suggest" in qn or "best" in qn or "good" in qn
    ):
        return True
    return False

def detect_affirmation(q: str) -> bool:
    qn = normalize_query(q)
    return qn in {"yes", "y", "yeah", "yep", "sure", "ok", "okay", "correct", "right"}

def is_out_of_scope(q: str) -> bool:
    q = q.lower()
    triggers = [
        "medical advice", "diagnose", "diagnosis", "treatment",
        "legal advice", "is it legal", "can i sue",
        "financial advice", "should i invest",
        "forecast", "prediction", "future price",
        "train a model", "fine-tune", "finetune",
        "hipaa", "patient record", "medical record",
        "ssn", "social security", "diagnosis code",
        "scrape", "scraping", "crawl", "crawling",
        "reddit", "twitter", "unverified source"
    ]
    return any(t in q for t in triggers)

# =========================================================
# SLOT PLANNER (returns slot + ask-back question)
# =========================================================
def plan_missing_slot(query: str):
    q = query.lower()

    if "copay" in q and not any(x in q for x in ["aetna","anthem","hip","gk","nyc","ppo","epo","hmo"]):
        return ("awaiting_plan_name",
                "Which plan are you enrolled in (or comparing)? (e.g., Aetna EPO, Anthem PPO, HIP HMO)")

    if any(x in q for x in ["add", "enroll", "include"]) and any(x in q for x in ["child", "dependent", "son", "daughter"]) \
       and not any(x in q for x in ["birth", "adoption", "qualifying", "life event", "open enrollment"]):
        return ("awaiting_timing",
                "Is this during open enrollment, or a qualifying life event (birth/adoption)? What is the event date?")

    return (None, None)

# =========================================================
# EVIDENCE BUILDER — schema-safe (meta or top-level)
# =========================================================
def build_evidence(results, max_evidence=2, max_lines=2):
    ev = []
    for r in results[:max_evidence]:
        lines = [ln.strip() for ln in r.get("text", "").splitlines() if ln.strip()]
        snippet = " ".join(lines[:max_lines])

        meta = r.get("meta", {}) or {}
        key = r.get("key") or meta.get("key") or r.get("source") or meta.get("source") or "unknown_chunk"

        ev.append(f"- {key} (score={r.get('score', 0.0):.3f}): {snippet[:220]}")
    return "\n".join(ev)

# =========================================================
# PLAN CATALOG EXTRACTION
# =========================================================
def _chunk_type(r: dict) -> str:
    meta = r.get("meta", {}) or {}
    return r.get("chunk_type") or meta.get("chunk_type") or ""

def is_plan_options_query(q: str) -> bool:
    q = q.lower()
    phrases = [
        "plan options", "plans available", "list all plans", "which plans", "plan choices",
        "available plans", "what plans", "plan list", "options available"
    ]
    return any(p in q for p in phrases)

def answer_plan_options_deterministic(results):
    catalogs = [r for r in results if _chunk_type(r) == "plan_catalog"]
    if not catalogs:
        return None
    combined = "\n".join([c.get("text", "") for c in catalogs])
    plans = extract_plan_names_from_catalog(combined)  # must exist in your notebook
    if not plans:
        return None
    return "**Available plans found in the documents:**\n- " + "\n- ".join(plans)

# =========================================================
# CORE ANSWER FUNCTION (NEW NAME, ALWAYS RETURNS 5)
# =========================================================
def answer_with_intelligent_fallback_insurance_v3_from_results(query, results, threshold=0.40):
    # STRICT FALLBACK FIRST
    if is_personal_recommendation(query):
        return STRICT_FALLBACK_MSG, "", 0.0, "Strict Fallback (Personal Recommendation)", None

    if is_out_of_scope(query):
        return STRICT_FALLBACK_MSG, "", 0.0, "Strict Fallback (Out of Scope)", None

    # SLOT-AWARE ASK-BACK
    pending_slot, followup = plan_missing_slot(query)
    if followup:
        return followup, "", 0.0, "Ask-Back (Missing Details)", pending_slot

    # Score + evidence
    top_score = results[0]["score"] if results else 0.0
    evidence = build_evidence(results)

    # Deterministic plan extraction
    if is_plan_options_query(query):
        extracted = answer_plan_options_deterministic(results)
        if extracted:
            return extracted, evidence, top_score, "Plan Catalog Extraction", None

    # Confidence gate → fallback
    if top_score < threshold:
        return (
            "I can’t confirm that from the policy documents I have indexed. "
            "If you share your plan name (and timing: open enrollment vs qualifying event), I can narrow it down.",
            evidence,
            top_score,
            "Intelligent Fallback Used",
            None
        )

    # RAG answer
    messages = [
        {"role": "system", "content": (
            "You are an NYC Employee Health Insurance Assistant.\n"
            "Use ONLY the provided evidence.\n"
            "Never invent policy details or numbers.\n"
            "If evidence is insufficient, ask exactly ONE targeted follow-up.\n"
            "Keep the answer concise."
        )},
        {"role": "user", "content": (
            f"EVIDENCE:\n{evidence}\n\n"
            f"TOP_SIMILARITY_SCORE: {top_score:.3f}\n\n"
            f"USER_QUESTION: {query}\n\n"
            "TASK:\n- Answer using evidence only.\n"
            "- If missing key details, ask exactly ONE targeted follow-up question."
        )}
    ]

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.2,
        max_tokens=220
    )
    ans = resp.choices[0].message.content
    return ans, evidence, top_score, "RAG Answer", None

# =========================================================
# STATEFUL CHAT FN (RETURNS 5 + UPDATES pending_slot)
# =========================================================
def chat_fn_intelligent(q, state):
    state = state or {"last_question": "", "pending_slot": None}

    # Strict fallback before anything
    if is_personal_recommendation(q):
        return STRICT_FALLBACK_MSG, "", f"{0.0:.3f}", "Strict Fallback (Personal Recommendation)", state

    # If user says "yes" while waiting for a slot, do NOT treat it as a new query
    if detect_affirmation(q) and state.get("pending_slot"):
        slot = state["pending_slot"]
        if slot == "awaiting_plan_name":
            return (
                "Please share the plan name (e.g., Aetna EPO, Anthem PPO, HIP HMO).",
                "",
                f"{0.0:.3f}",
                "Ask-Back (Plan Name Needed)",
                state
            )
        if slot == "awaiting_timing":
            return (
                "Please confirm: open enrollment or qualifying life event? If life event, what is the date?",
                "",
                f"{0.0:.3f}",
                "Ask-Back (Timing Needed)",
                state
            )

    # Retrieval
    results = retrieve_insurance(q, k=4, min_score=0.22)  # must exist in your notebook

    # Core logic (5 returns)
    ans, evidence, score, mode, pending_slot = answer_with_intelligent_fallback_insurance_v3_from_results(
        q, results, threshold=0.40
    )

    state["last_question"] = q
    state["pending_slot"] = pending_slot

    return ans, evidence, f"{score:.3f}", mode, state

# =========================================================
# 6) ON-DEMAND AUDIO GENERATION (BUTTON)
# =========================================================
def generate_audio_from_answer(ans, voice_choice):
    if not ans or not str(ans).strip():
        return None

    voice_map = {
        "Alloy (Neutral)": "alloy",
        "Echo (Male)": "echo",
        "Fable (British Male)": "fable",
        "Onyx (Deep Male)": "onyx",
        "Nova (Female)": "nova",
        "Shimmer (Soft Female)": "shimmer"
    }

    response = client.audio.speech.create(
        model="tts-1",
        voice=voice_map.get(voice_choice, "nova"),
        input=ans
    )
    audio_path = "response.mp3"
    response.stream_to_file(audio_path)
    return audio_path

# =========================================================
# 7) STYLING (UNCHANGED)
# =========================================================
HEALTH_CSS = """
:root{
  --brand:#0ea5a8;
  --brand2:#2563eb;
  --soft:#ecfeff;
  --card:#ffffff;
  --border:rgba(15, 23, 42, .08);
  --text:#0f172a;
  --muted:#475569;
  --danger:#ef4444;
  --shadow: 0 10px 30px rgba(2, 6, 23, .08);
  --radius:18px;
}
.gradio-container{
  background: linear-gradient(180deg, var(--soft), #f8fafc 45%, #ffffff 100%);
  color: var(--text);
}
#app_title h1{ font-weight: 800; letter-spacing: -0.02em; margin-bottom: .25rem; }
#app_title p{ color: var(--muted); margin-top: 0; }
.card{
  background: var(--card) !important;
  border: 1px solid var(--border) !important;
  border-radius: var(--radius) !important;
  box-shadow: var(--shadow);
}
.pill{
  display:inline-flex;
  gap:.4rem;
  align-items:center;
  padding:.35rem .65rem;
  border-radius: 999px;
  border: 1px solid var(--border);
  background: rgba(255,255,255,.75);
  color: var(--muted);
  font-size: .85rem;
}
#ask_btn{
  background: linear-gradient(90deg, var(--brand), var(--brand2)) !important;
  border: none !important;
  color: white !important;
  font-weight: 700 !important;
  border-radius: 14px !important;
  padding: .75rem 1rem !important;
}
#clear_btn{
  border: 1px solid var(--border) !important;
  border-radius: 14px !important;
  font-weight: 600 !important;
}
textarea, input{
  border-radius: 14px !important;
  border: 1px solid var(--border) !important;
}
label{
  font-weight: 650 !important;
  color: var(--text) !important;
}
.small_note{
  color: var(--muted);
  font-size: .90rem;
}
.kpi{
  display:flex;
  gap:.75rem;
  flex-wrap: wrap;
  margin-top: .5rem;
}
.kpi .box{
  border: 1px solid var(--border);
  background: rgba(255,255,255,.8);
  border-radius: 14px;
  padding: .6rem .8rem;
  min-width: 160px;
}
.kpi .box .t{ color: var(--muted); font-size: .8rem; }
.kpi .box .v{ font-size: 1.05rem; font-weight: 750; margin-top: .15rem; }
"""

theme = gr.themes.Soft(
    primary_hue="cyan",
    secondary_hue="blue",
    neutral_hue="slate",
    radius_size="lg",
    font="Inter, ui-sans-serif, system-ui, sans-serif"
)

# =========================================================
# 8) GRADIO APP (STATEFUL)
# =========================================================
with gr.Blocks(theme=theme, css=HEALTH_CSS, title="NYC Employee Health Insurance Assistant") as demo:
    gr.HTML("""
    <div id="app_title">
      <h1>NYC Employee Health Insurance Assistant</h1>
      <p>Policy-grounded answers with intelligent fallback when documents don’t support a definitive response.</p>
      <span class="pill">RAG • Evidence-first • No hallucinations</span>
    </div>
    """)
    gr.Markdown("**🔒 Session isolation:** This chat history is stored only in your current session and clears when you click Clear or refresh.")

    history_state = gr.State([])
    slot_state = gr.State({"last_question": "", "pending_slot": None})

    with gr.Row():
        with gr.Column(scale=7):
            with gr.Group(elem_classes="card"):
                inp = gr.Textbox(
                    label="Ask a benefits question",
                    placeholder="Examples: What are my plan options? How do I add my child? What is a copay?",
                    lines=2
                )
                voice = gr.Dropdown(
                    choices=["Alloy (Neutral)", "Echo (Male)", "Fable (British Male)",
                             "Onyx (Deep Male)", "Nova (Female)", "Shimmer (Soft Female)"],
                    value="Nova (Female)",
                    label="Voice"
                )
                with gr.Row():
                    ask_btn = gr.Button("Get Answer", elem_id="ask_btn")
                    tts_btn = gr.Button("Generate Voice")
                    clear_btn = gr.Button("Clear", elem_id="clear_btn")

                gr.Markdown(
                    "<div class='small_note'>Tip: For cost questions, include your plan name (e.g., Aetna EPO / NYCE PPO) for higher accuracy.</div>"
                )

        with gr.Column(scale=5):
            with gr.Group(elem_classes="card"):
                gr.Markdown("### Retrieval Diagnostics")
                score_box = gr.Textbox(label="Top Similarity Score", interactive=False)
                mode_box  = gr.Textbox(label="Response Mode", interactive=False)
                gr.HTML("""
                <div class="kpi">
                  <div class="box"><div class="t">Quality Gate</div><div class="v">Thresholded Retrieval</div></div>
                  <div class="box"><div class="t">Safety</div><div class="v">Context-only Answers</div></div>
                </div>
                """)

    with gr.Row():
        with gr.Column(scale=7):
            with gr.Group(elem_classes="card"):
                out = gr.Textbox(label="Answer", lines=6, interactive=False)
                audio_out = gr.Audio(label="🔊 Voice", autoplay=False)

        with gr.Column(scale=5):
            with gr.Accordion("Evidence / Retrieved Context (for transparency)", open=False):
                ctx = gr.Textbox(label="Evidence (Top Chunks)", lines=14, interactive=False)

    def clear_all():
        return "", "Nova (Female)", "", "", "", "", None, [], {"last_question": "", "pending_slot": None}

    def answer_and_clear_audio_multiturn(q, history, slot_state_in):
        ans, evidence, score, mode, slot_state_out = chat_fn_intelligent(q, slot_state_in)
        history.append((q, ans))
        return ans, evidence, score, f"{mode} + Memory", None, history, slot_state_out

    inp.submit(
        answer_and_clear_audio_multiturn,
        [inp, history_state, slot_state],
        [out, ctx, score_box, mode_box, audio_out, history_state, slot_state]
    )
    ask_btn.click(
        answer_and_clear_audio_multiturn,
        [inp, history_state, slot_state],
        [out, ctx, score_box, mode_box, audio_out, history_state, slot_state]
    )

    tts_btn.click(generate_audio_from_answer, [out, voice], [audio_out])

    clear_btn.click(
        clear_all,
        None,
        [inp, voice, out, ctx, score_box, mode_box, audio_out, history_state, slot_state]
    )

/tmp/ipython-input-3995111090.py:346: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css=HEALTH_CSS, title="NYC Employee Health Insurance Assistant") as demo:
/tmp/ipython-input-3995111090.py:346: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css=HEALTH_CSS, title="NYC Employee Health Insurance Assistant") as demo:


In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dada9a8494b27295a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dada9a8494b27295a1.gradio.live
